In [28]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import absl.logging
import tensorflow as tf
from keras.callbacks import EarlyStopping


def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))\

early_stopping = EarlyStopping(monitor='val_loss', patience=50, mode='auto')

absl.logging.set_verbosity(absl.logging.ERROR) # .h5 에러메세지 무시

mat_file = scipy.io.loadmat("data/PTSDvsHC.mat")

dataset = mat_file['dataset'] 
# 90개의 데이터, 62개의 전극 채널, 1001의 타임스텝 - 1001씩 이미 에포킹을 한 상태이며, 그에따른 평균을 낸 것임(에포킹에 대해)

subject_test_acc = []

X = dataset
X = X.reshape(90, 1001, 62, 1)
y = mat_file['Y']
y = to_categorical(y, num_classes = 2)

hc_indices = list(range(0, 51))
pg_indices = list(range(51, 90))

fold_val_index_list = []
fold_accuracy = []

input_shape = (1001, 62, 1)

for i in range(len(dataset)): # loso
    test_index = [i]

    hc_val = random.sample([idx for idx in hc_indices if idx not in test_index], 5)
    pg_val = random.sample([idx for idx in pg_indices if idx not in test_index], 5)

    validation_indices = hc_val + pg_val

    training_indices = [idx for idx in range(90) if idx not in test_index + validation_indices]

    fold_val_index_list.append({
        'fold' : i, 
        'test_index' : test_index, 
        'validation_indices' : validation_indices
    })
    
    X_train = X[training_indices, :,  :]
    y_train = y[training_indices, :]
    X_test = X[test_index, :,  :]
    y_test = y[test_index, :]
    X_val = X[validation_indices, :,  :]
    y_val = y[validation_indices, :]

    # print(f"X_train_shape : {X_train.shape}")
    # print(f"X_test_shape : {X_test.shape}")
    # print(f"y_train_shape : {y_train.shape}")
    # print(f"y_test_shape : {y_test.shape}")
    # print(f"X_validation_shape : {X_val.shape}")
    # print(f"y_validation_shape : {y_val.shape}")
    
    model_input = layers.Input(shape = input_shape)
    br1 = layers.Conv2D(32, (50, 1), padding = 'same')(model_input) # temporal filter
    br1 = layers.Activation(mish)(br1) # result1_matrix, shape=(1001, 62, 32)
    # print(br1.shape)

    input_shape_tr = layers.Permute((2, 1, 3))(model_input) # (62, 1001, 1)
    br2 = layers.Conv2D(32, (62, 1), padding = 'same')(input_shape_tr) # shape=(62, 1001, 32)

    # br1, br2 matrix multiply
    br1 = layers.Permute((3, 1, 2))(br1) # (32, 1001, 62)
    br2 = layers.Permute((3, 1, 2))(br2) # (32, 62, 1001)
    br_mul = layers.Lambda(lambda x: tf.matmul(x[0], x[1]))([br2, br1])
    # print(br_mul.shape)  # (32, 62, 62)

    # branch matrix multiply convolution
    br_mul = layers.Permute((2, 3, 1))(br_mul) # (62, 62, 32) <- 풀링하기 위함
    # 여기에 활성화 함수 X -> 선형성 때문
    br_mul = layers.Conv2D(16, (3, 3), padding = 'same')(br_mul) # ######################<- 8로 수정 예정
    br_mul = layers.Activation(mish)(br_mul)
    br_mul = layers.GlobalAveragePooling2D()(br_mul)
    end = layers.Flatten()(br_mul)

    prediction = layers.Dense(2, activation = 'softmax')(end)
    model = keras.Model(inputs = model_input, outputs = prediction)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs=15, batch_size=1, verbose=2, callbacks=[early_stopping])
    predictions = model.predict(X_test)
    
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(y_test, axis=1)

    acc = accuracy_score(predicted_classes, actual_classes)
    fold_accuracy.append(acc)

    # model.summary()
    
    print(f"fold {i} test accuracy : {acc}\n")

    subject_test_acc.append(acc)
scipy.io.savemat('fold_val_indices.mat', {'fold_val_index_list': fold_val_index_list})
print(f"loso mean accuracy : {np.mean(subject_test_acc)}")

Epoch 1/15
79/79 - 3s - 43ms/step - accuracy: 0.5696 - loss: 30.1902 - val_accuracy: 0.6000 - val_loss: 1.0522
Epoch 2/15
79/79 - 2s - 28ms/step - accuracy: 0.6203 - loss: 1.0170 - val_accuracy: 0.8000 - val_loss: 0.6622
Epoch 3/15
79/79 - 2s - 29ms/step - accuracy: 0.6203 - loss: 0.6087 - val_accuracy: 0.7000 - val_loss: 0.7367
Epoch 4/15
79/79 - 2s - 28ms/step - accuracy: 0.6076 - loss: 0.5478 - val_accuracy: 0.8000 - val_loss: 0.4853
Epoch 5/15
79/79 - 2s - 28ms/step - accuracy: 0.6962 - loss: 0.4442 - val_accuracy: 0.7000 - val_loss: 0.4764
Epoch 6/15


KeyboardInterrupt: 

In [2]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import absl.logging
import tensorflow as tf
from keras.callbacks import EarlyStopping


def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))\

early_stopping = EarlyStopping(monitor='val_loss', patience=50, mode='auto')

absl.logging.set_verbosity(absl.logging.ERROR) # .h5 에러메세지 무시

mat_file = scipy.io.loadmat("data/PTSDvsHC.mat")

dataset = mat_file['dataset'] 
# 90개의 데이터, 62개의 전극 채널, 1001의 타임스텝 - 1001씩 이미 에포킹을 한 상태이며, 그에따른 평균을 낸 것임(에포킹에 대해)

subject_test_acc = []

X = dataset
X = X.reshape(90, 1001, 62, 1)
y = mat_file['Y']
y = to_categorical(y, num_classes = 2)

hc_indices = list(range(0, 51))
pg_indices = list(range(51, 90))

fold_val_index_list = []
fold_accuracy = []

input_shape = (1001, 62, 1)

for i in range(len(dataset)): # loso
    test_index = [i]

    hc_val = random.sample([idx for idx in hc_indices if idx not in test_index], 5)
    pg_val = random.sample([idx for idx in pg_indices if idx not in test_index], 5)

    validation_indices = hc_val + pg_val

    training_indices = [idx for idx in range(90) if idx not in test_index + validation_indices]

    fold_val_index_list.append({
        'fold' : i, 
        'test_index' : test_index, 
        'validation_indices' : validation_indices
    })
    
    X_train = X[training_indices, :,  :]
    y_train = y[training_indices, :]
    X_test = X[test_index, :,  :]
    y_test = y[test_index, :]
    X_val = X[validation_indices, :,  :]
    y_val = y[validation_indices, :]

    # print(f"X_train_shape : {X_train.shape}")
    # print(f"X_test_shape : {X_test.shape}")
    # print(f"y_train_shape : {y_train.shape}")
    # print(f"y_test_shape : {y_test.shape}")
    # print(f"X_validation_shape : {X_val.shape}")
    # print(f"y_validation_shape : {y_val.shape}")
    
    model_input = layers.Input(shape = input_shape)
    br1 = layers.Conv2D(32, (50, 1), padding = 'same')(model_input) # temporal filter
    br1 = layers.Activation(mish)(br1) # result1_matrix, shape=(1001, 62, 32)
    # print(br1.shape)

    input_shape_tr = layers.Permute((2, 1, 3))(model_input) # (62, 1001, 1)
    br2 = layers.Conv2D(32, (62, 1), padding = 'same')(input_shape_tr) # shape=(62, 1001, 32)

    # br1, br2 matrix multiply
    br1 = layers.Permute((1, 3, 2))(br1) # 1001, 62, 32
    br2 = layers.Permute((2, 1, 3))(br2) # 1001,
    br_mul = layers.Lambda(lambda x: tf.matmul(x[0], x[1]))([br2, br1]) # 1001 62 62

    # branch matrix multiply convolution
    br_mul = layers.Permute((2, 3, 1))(br_mul) # (62, 62, 1001)
    print(br_mul.shape)
    # 여기에 활성화 함수 X -> 선형성 때문
    end = layers.Flatten()(br_mul)

    prediction = layers.Dense(2, activation = 'softmax')(end)
    model = keras.Model(inputs = model_input, outputs = prediction)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs=40, batch_size=1, verbose=2, callbacks=[early_stopping])
    predictions = model.predict(X_test)
    
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(y_test, axis=1)

    acc = accuracy_score(predicted_classes, actual_classes)
    fold_accuracy.append(acc)

    # model.summary()
    
    print(f"fold {i} test accuracy : {acc}\n")

    subject_test_acc.append(acc)
scipy.io.savemat('fold_val_indices.mat', {'fold_val_index_list': fold_val_index_list})
print(f"loso mean accuracy : {np.mean(subject_test_acc)}")

(None, 62, 62, 1001)
Epoch 1/40
79/79 - 7s - 90ms/step - accuracy: 0.5443 - loss: 1550.5325 - val_accuracy: 0.5000 - val_loss: 8434.1992
Epoch 2/40
79/79 - 6s - 78ms/step - accuracy: 0.6962 - loss: 2012.6993 - val_accuracy: 0.7000 - val_loss: 4061.5007
Epoch 3/40
79/79 - 6s - 77ms/step - accuracy: 0.8987 - loss: 155.9801 - val_accuracy: 0.7000 - val_loss: 3941.7456
Epoch 4/40
79/79 - 6s - 77ms/step - accuracy: 0.9367 - loss: 52.1283 - val_accuracy: 0.8000 - val_loss: 1757.9121
Epoch 5/40
79/79 - 6s - 78ms/step - accuracy: 0.9367 - loss: 130.8107 - val_accuracy: 0.7000 - val_loss: 4574.0161
Epoch 6/40
79/79 - 6s - 78ms/step - accuracy: 0.8608 - loss: 482.9201 - val_accuracy: 0.7000 - val_loss: 1935.5505
Epoch 7/40
79/79 - 6s - 77ms/step - accuracy: 0.8354 - loss: 1612.8710 - val_accuracy: 0.8000 - val_loss: 6323.9795
Epoch 8/40
79/79 - 6s - 77ms/step - accuracy: 0.8987 - loss: 601.2427 - val_accuracy: 0.7000 - val_loss: 6096.5396
Epoch 9/40
79/79 - 6s - 77ms/step - accuracy: 0.8481 - lo

KeyboardInterrupt: 

In [3]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 1001, 62, 1)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_4 (Permute)           │ (None, 62, 1001, 1)       │               0 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 1001, 62, 32)      │           1,632 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 62, 1001, 32)      │           2,016 │ permute_4[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 1001, 62, 32)      │               0 │ conv2d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_6 (Permute)           │ (None, 1001, 62, 32)      │               0 │ conv2d_3[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_5 (Permute)           │ (None, 1001, 32, 62)      │               0 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda_1 (Lambda)             │ (None, 1001, 62, 62)      │               0 │ permute_6[0][0],           │
│                               │                           │                 │ permute_5[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_7 (Permute)           │ (None, 62, 62, 1001)      │               0 │ lambda_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 3847844)           │               0 │ permute_7[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 2)                 │       7,695,690 │ flatten_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 23,098,016 (88.11 MB)

 Trainable params: 7,699,338 (29.37 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 15,398,678 (58.74 MB)